In [1]:
import pandas as pd
import ee

ee.Authenticate()
ee.Initialize()

/Users/jp/miniforge3/envs/id2223/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.15) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
obs = pd.read_csv('./data/beetle/artportalen_final.csv')
obs["Month"] = pd.to_datetime(obs['Date']).dt.to_period("M").dt.to_timestamp()

obs.drop(columns=['Kommun', 'Lan','Quantity','Date'], inplace=True)

obs["row_id"] = obs.index.astype(int)
print(f"Obs has {len(obs)} datapoints")
obs.head(5)

Obs has 1022 datapoints


,Lat,Lon,Month,row_id
0,64.024023,20.650910,2018-09-01,0
1,56.729677,15.956413,2018-09-01,1
2,55.614954,14.276141,2018-09-01,2
3,61.714395,17.372628,2018-06-01,3
4,56.730931,15.906116,2018-09-01,4


# Download NDVI data
We will use Google Earth Engine to get this data

In [ ]:
def df_to_ee_points(df):
    """
    Convert df rows to EE points (Earth Engine format) and add metadata (row id and date of the month)
    """
    features = []

    for _, row in obs.iterrows():
        features.append(
            ee.Feature(
                ee.Geometry.Point(row["Lon"], row["Lat"]),
                {
                    "row_id": int(row["row_id"]),
                    "month": row["Month"].strftime("%Y-%m-01"),
                    "Lat": float(row["Lat"]),
                    "Lon": float(row["Lon"]),
                }
            )
        )

    return ee.FeatureCollection(features)


In [18]:
def monthly_ndvi_image(dataset, year, month):
    """
    Select satellite image from modis 
    """
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    monthly = dataset.filterDate(start, end)

    return (
        ee.Image(
            ee.Algorithms.If(
                monthly.size().gt(0),
                monthly.mean().multiply(0.0001).rename("NDVI"),
                ee.Image().rename("NDVI")
            )
        )
        .set("year", year)
        .set("month", month)
    )

def extract_monthly_ndvi(dataset, obs_dataset, start_year=2000, end_year=2025):
    records = []

    points_fc = df_to_ee_points(obs_dataset)

    # Generate all months
    months = obs_dataset["Month"].drop_duplicates()

    for m in months:
        year, month = m.year, m.month
        img = monthly_ndvi_image(dataset, year, month)

        sampled = img.sampleRegions(
            collection=points_fc,
            scale=250,
            geometries=False
        )

        info = sampled.getInfo()
        for f in info["features"]:
            props = f["properties"]
            records.append({
                "row_id": props["row_id"],
                "Lat": props.get("Lat"),
                "Lon": props.get("Lon"),
                "Month": m,
                "NDVI": props.get("NDVI")
            })

    return pd.DataFrame(records)


In [20]:
# MODIS is a dataset containing a time series of satellite images 
modis = ee.ImageCollection("MODIS/061/MOD13Q1").select("NDVI")
ndvi_df = extract_monthly_ndvi(modis, obs)

ndvi_df

,row_id,Lat,Lon,Month,NDVI
0,0,64.024023,20.650910,2018-09-01,0.60930
1,1,56.729677,15.956413,2018-09-01,0.60710
2,2,55.614954,14.276141,2018-09-01,0.72935
3,3,61.714395,17.372628,2018-09-01,0.72515
4,4,56.730931,15.906116,2018-09-01,0.69140
...,...,...,...,...,...
87401,1017,61.006044,15.190126,2025-11-01,0.47200
87402,1018,57.858032,15.090049,2025-11-01,0.58830
87403,1019,59.993022,18.877981,2025-11-01,0.31505
87404,1020,58.913920,14.528770,2025-11-01,0.61540


In [ ]:
if False: ndvi_df.to_csv('./data/ndvi/ndvi_raw.csv', index=False)


## 2. Create features

### 2.1 Anomalies: compare NDVI for a month vs. climatology (average of that month for previous years)

In [22]:
ndvi_df["Month_num"] = ndvi_df["Month"].dt.month

ndvi_climatology = (
    ndvi_df
    .groupby(["Lat", "Lon", "Month_num"], as_index=False)
    .agg(NDVI_clim=("NDVI", "mean"))
)
ndvi_climatology

,Lat,Lon,Month_num,NDVI_clim
0,55.542513,13.321149,1,0.426740
1,55.542513,13.321149,2,0.644343
2,55.542513,13.321149,3,0.554850
3,55.542513,13.321149,4,0.621519
4,55.542513,13.321149,5,0.757869
...,...,...,...,...
10428,67.891230,20.741564,7,0.786956
10429,67.891230,20.741564,8,0.769225
10430,67.891230,20.741564,9,0.673638
10431,67.891230,20.741564,10,0.363193


In [23]:
ndvi_features = ndvi_df.merge(ndvi_climatology, on=["Lat", "Lon", "Month_num"],how="left")
ndvi_features

,row_id,Lat,Lon,Month,NDVI,Month_num,NDVI_clim
0,0,64.024023,20.650910,2018-09-01,0.60930,9,0.623756
1,1,56.729677,15.956413,2018-09-01,0.60710,9,0.600225
2,2,55.614954,14.276141,2018-09-01,0.72935,9,0.694438
3,3,61.714395,17.372628,2018-09-01,0.72515,9,0.713806
4,4,56.730931,15.906116,2018-09-01,0.69140,9,0.724681
...,...,...,...,...,...,...,...
87401,1017,61.006044,15.190126,2025-11-01,0.47200,11,0.505800
87402,1018,57.858032,15.090049,2025-11-01,0.58830,11,0.512650
87403,1019,59.993022,18.877981,2025-11-01,0.31505,11,0.376812
87404,1020,58.913920,14.528770,2025-11-01,0.61540,11,0.567388


In [24]:
ndvi_features["NDVI_anom"] = ndvi_features["NDVI"] - ndvi_features["NDVI_clim"]
ndvi_features

,row_id,Lat,Lon,Month,NDVI,Month_num,NDVI_clim,NDVI_anom
0,0,64.024023,20.650910,2018-09-01,0.60930,9,0.623756,-0.014456
1,1,56.729677,15.956413,2018-09-01,0.60710,9,0.600225,0.006875
2,2,55.614954,14.276141,2018-09-01,0.72935,9,0.694438,0.034913
3,3,61.714395,17.372628,2018-09-01,0.72515,9,0.713806,0.011344
4,4,56.730931,15.906116,2018-09-01,0.69140,9,0.724681,-0.033281
...,...,...,...,...,...,...,...,...
87401,1017,61.006044,15.190126,2025-11-01,0.47200,11,0.505800,-0.033800
87402,1018,57.858032,15.090049,2025-11-01,0.58830,11,0.512650,0.075650
87403,1019,59.993022,18.877981,2025-11-01,0.31505,11,0.376812,-0.061762
87404,1020,58.913920,14.528770,2025-11-01,0.61540,11,0.567388,0.048013


### 2.2 Lagged features

In [25]:
ndvi_features = ndvi_features.sort_values(["Lat", "Lon", "Month"])

MAX_LAG = 2

for lag in range(1, MAX_LAG + 1):
    ndvi_features[f"NDVI_lag{lag}"] = (
        ndvi_features
        .groupby(["Lat", "Lon"])["NDVI"]
        .shift(lag)
    )

    ndvi_features[f"NDVI_anom_lag{lag}"] = (
        ndvi_features
        .groupby(["Lat", "Lon"])["NDVI_anom"]
        .shift(lag)
    )

ndvi_features


,row_id,Lat,Lon,Month,NDVI,Month_num,NDVI_clim,NDVI_anom,NDVI_lag1,NDVI_anom_lag1,NDVI_lag2,NDVI_anom_lag2
8466,550,55.542513,13.321149,2018-02-01,0.73495,2,0.644343,0.090607,NaN,NaN,NaN,NaN
2576,550,55.542513,13.321149,2018-03-01,0.42985,3,0.554850,-0.125000,0.73495,0.090607,NaN,NaN
6635,550,55.542513,13.321149,2018-04-01,0.75935,4,0.621519,0.137831,0.42985,-0.125000,0.73495,0.090607
5618,550,55.542513,13.321149,2018-05-01,0.86870,5,0.757869,0.110831,0.75935,0.137831,0.42985,-0.125000
1560,550,55.542513,13.321149,2018-06-01,0.87220,6,0.803556,0.068644,0.86870,0.110831,0.75935,0.137831
...,...,...,...,...,...,...,...,...,...,...,...,...
85503,124,67.891230,20.741564,2025-07-01,0.82175,7,0.786956,0.034794,0.74490,-0.038844,0.64265,0.196006
82463,124,67.891230,20.741564,2025-08-01,0.80555,8,0.769225,0.036325,0.82175,0.034794,0.74490,-0.038844
83479,124,67.891230,20.741564,2025-09-01,0.74025,9,0.673638,0.066613,0.80555,0.036325,0.82175,0.034794
84493,124,67.891230,20.741564,2025-10-01,0.59580,10,0.363193,0.232607,0.74025,0.066613,0.80555,0.036325


In [27]:
if False: ndvi_features.to_csv('./data/ndvi/ndvi_features.csv', index=False)

## Map back to beetle dataset schema (1 row = 1 observation, 1 coordinate, 1 day)

In [ ]:
ndvi_features = pd.read_csv('./data/ndvi/ndvi_features.csv')
ndvi_features["Month"] = pd.to_datetime(ndvi_features['Month'])

ndvi_features

,row_id,Lat,Lon,Month,NDVI,Month_num,NDVI_clim,NDVI_anom,NDVI_lag1,NDVI_anom_lag1,NDVI_lag2,NDVI_anom_lag2
0,550,55.542513,13.321149,2018-02-01,0.73495,2,0.644343,0.090607,NaN,NaN,NaN,NaN
1,550,55.542513,13.321149,2018-03-01,0.42985,3,0.554850,-0.125000,0.73495,0.090607,NaN,NaN
2,550,55.542513,13.321149,2018-04-01,0.75935,4,0.621519,0.137831,0.42985,-0.125000,0.73495,0.090607
3,550,55.542513,13.321149,2018-05-01,0.86870,5,0.757869,0.110831,0.75935,0.137831,0.42985,-0.125000
4,550,55.542513,13.321149,2018-06-01,0.87220,6,0.803556,0.068644,0.86870,0.110831,0.75935,0.137831
...,...,...,...,...,...,...,...,...,...,...,...,...
87401,124,67.891230,20.741564,2025-07-01,0.82175,7,0.786956,0.034794,0.74490,-0.038844,0.64265,0.196006
87402,124,67.891230,20.741564,2025-08-01,0.80555,8,0.769225,0.036325,0.82175,0.034794,0.74490,-0.038844
87403,124,67.891230,20.741564,2025-09-01,0.74025,9,0.673638,0.066613,0.80555,0.036325,0.82175,0.034794
87404,124,67.891230,20.741564,2025-10-01,0.59580,10,0.363193,0.232607,0.74025,0.066613,0.80555,0.036325


In [45]:
ndvi_final = obs.merge(ndvi_features, on=["Lat", "Lon", "row_id", "Month"], how="left")
ndvi_final.drop(columns=['row_id', 'Month_num'], inplace=True)
ndvi_final

,Lat,Lon,Month,NDVI,NDVI_clim,NDVI_anom,NDVI_lag1,NDVI_anom_lag1,NDVI_lag2,NDVI_anom_lag2
0,64.024023,20.650910,2018-09-01,0.60930,0.623756,-0.014456,0.61225,-0.076781,0.65995,-0.043456
1,56.729677,15.956413,2018-09-01,0.60710,0.600225,0.006875,0.61515,0.038256,0.61625,-0.026462
2,55.614954,14.276141,2018-09-01,0.72935,0.694438,0.034912,0.69880,0.007531,0.67365,-0.029069
3,61.714395,17.372628,2018-06-01,0.74255,0.774188,-0.031637,0.70265,-0.004019,0.41235,-0.223112
4,56.730931,15.906116,2018-09-01,0.69140,0.724681,-0.033281,0.70405,-0.047056,0.69195,-0.057462
...,...,...,...,...,...,...,...,...,...,...
1017,61.006044,15.190126,2024-10-01,0.68830,0.755243,-0.066943,0.78480,0.046519,0.76730,0.024381
1018,57.858032,15.090049,2025-04-01,0.65635,0.671231,-0.014881,0.59145,-0.006969,0.61105,0.069121
1019,59.993022,18.877981,2025-11-01,0.31505,0.376812,-0.061762,0.63320,0.056314,0.78280,0.038856
1020,58.913920,14.528770,2024-09-01,0.77580,0.783181,-0.007381,0.78690,0.010412,0.81225,0.015844


In [47]:
if False: ndvi_final.to_csv('./data/ndvi/ndvi_final.csv', index=False)